In [89]:
### Preprocessing

In [114]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

# Chemin vers le dossier contenant les fichiers CSV
dossier = 'database/'

# Liste pour stocker les données de tous les fichiers CSV
donnees_totales = []

# Parcours de tous les fichiers dans le dossier
for fichier in os.listdir(dossier):
    if fichier.endswith('.csv'):
        chemin_fichier = os.path.join(dossier, fichier)
        # Lecture du fichier CSV et ajout des données à la liste
        donnees = pd.read_csv(chemin_fichier)
        donnees_totales.append(donnees)

# Concaténation de toutes les données en un seul DataFrame
donnees_combinees = pd.concat(donnees_totales, ignore_index=True)

donnees_combinees['year'] = donnees_combinees['date'].apply(lambda x: int(x.split('-')[0]))
donnees_combinees['month'] = donnees_combinees['date'].apply(lambda x: int(x.split('-')[1]))
donnees_combinees['day'] = donnees_combinees['date'].apply(lambda x: int(x.split('-')[2]))


# Chemin vers le dossier contenant les fichiers CSV
dossier = 'database_2/'

# Liste pour stocker les données de tous les fichiers CSV
donnees_totales = []

# Parcours de tous les fichiers dans le dossier
for fichier in os.listdir(dossier):
    if fichier.endswith('.csv'):
        chemin_fichier = os.path.join(dossier, fichier)
        # Lecture du fichier CSV et ajout des données à la liste
        donnees = pd.read_csv(chemin_fichier)
        donnees_totales.append(donnees)

# Concaténation de toutes les données en un seul DataFrame
df_new = pd.concat(donnees_totales, ignore_index=True)

df_test = pd.merge(df_new, donnees_combinees, on='Country')

df_try = df_test[["Country", "Density\n(P/Km2)", "Land Area(Km2)", "Fertility Rate", "Infant mortality", "Life expectancy", "Population_y", "Latitude_x", "Longitude_x", "year", "month", "day"]]

label_encoder = LabelEncoder()

df_try['country_encoded'] = label_encoder.fit_transform(df_try['Country'])
df_try = df_try.dropna()
df_try['Land Area(Km2)'] = df_try['Land Area(Km2)'].str.replace(',', '').astype(float)
df_try['Density\n(P/Km2)'] = df_try['Density\n(P/Km2)'].str.replace(',', '').astype(float)

X = df_try[["Density\n(P/Km2)", "Land Area(Km2)", "Fertility Rate", "Infant mortality", "Life expectancy", "Latitude_x", "Longitude_x", "year", "month", "day", "country_encoded"]].values
#X = df_try[["Fertility Rate", "Life expectancy", "Latitude_x", "Longitude_x", "year", "month", "day", "country_encoded"]]
y = df_try["Population_y"].values.reshape(-1, 1)
train_ratio = 0.8
train_size = int(len(X) * train_ratio)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


# Set prediction data
df_pred = df_try[["Density\n(P/Km2)", "Land Area(Km2)", "Fertility Rate", "Infant mortality", "Life expectancy", "Latitude_x", "Longitude_x", "year", "month", "day", "country_encoded"]]
#df_pred = df_try[["Fertility Rate", "Life expectancy", "Latitude_x", "Longitude_x", "year", "month", "day", "country_encoded"]]
unique_combinations = df_pred.groupby(['Latitude_x', 'Longitude_x']).first().reset_index()
unique_combinations["year"] = 2025
unique_combinations["month"] = 1
unique_combinations["day"] = 1
X_pred = unique_combinations.values

C:\Users\jujuc\AppData\Local\Temp\ipykernel_1912\555554550.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_try['country_encoded'] = label_encoder.fit_transform(df_try['Country'])


In [91]:
### Prédictions KERAS

In [115]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape = (X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu', input_shape = 100),
    tf.keras.layers.Dense(1)  # Couche de sortie sans activation pour une régression
])

# Compiler le modèle
model.compile(optimizer='adam', loss='mean_squared_error')

# Entraîner le modèle
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

# Évaluer le modèle sur l'ensemble de test
loss = model.evaluate(X_test, y_test)
print("Loss sur l'ensemble de test:", loss)

# Faire des prédictions sur l'ensemble de test
predictions_1 = model.predict(X_pred)

# Afficher quelques prédictions
for i in range(10):
    print("Vraie valeur:", y[i][0], "| Prédiction:", predictions_1[i][0])

Epoch 1/50


C:\Users\jujuc\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


170/170 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 28092552154447872.0000 - val_loss: 145566122115072.0000
Epoch 2/50
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 25062701835223040.0000 - val_loss: 13657240087035904.0000
Epoch 3/50
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 25950583244455936.0000 - val_loss: 38333540894507008.0000
Epoch 4/50
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 21433868934447104.0000 - val_loss: 52098352732438528.0000
Epoch 5/50
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 20499266870968320.0000 - val_loss: 54804212193689600.0000
Epoch 6/50
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 23463631823831040.0000 - val_loss: 50716790602334208.0000
Epoch 7/50
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 26060613864128512.0000 - val_loss: 48291671678386176.0000
Epoch 8/50
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 20931490904801280.0000 - val_loss: 51457608036384768.0000
Epoch 9/50
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 212315394

In [118]:
df_pred_1 = unique_combinations.copy()
df_pred_1['predictions'] = predictions_1
df_pred_1['predictions'] = (df_pred_1['predictions']/10).astype("int64")
df_merged_1 = pd.merge(df_pred_1, df_try[['country_encoded', 'Country']], on='country_encoded', how='left').drop_duplicates().reset_index(drop=True)
df_merged_1['date'] = df_merged_1.apply(lambda row: '-'.join([str(row['year']), str(row['month']), str(row['day'])]), axis=1)

In [ ]:
#df_merged_1.to_csv(f'database_predict/pred.csv', index=False)

In [113]:
df_merged_1.loc[df_merged_1['Country'] == 'China']

,Latitude_x,Longitude_x,Density\n(P/Km2),Fertility Rate,Infant mortality,Life expectancy,year,month,day,country_encoded,predictions,Country
125,35.86166,104.195397,153.0,1.69,7.4,77.0,2025,1,1,34,109721,China


In [ ]:
### Prédictions SCRATCH

In [ ]:
import numpy as np

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # Initialisation des poids
        self.weights_input_hidden = np.random.randn(input_size, hidden_size)
        self.weights_hidden_output = np.random.randn(hidden_size, output_size)
        # Initialisation des biais
        self.bias_hidden = np.zeros(hidden_size)
        self.bias_output = np.zeros(output_size)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def rmse_loss(self, y_true, y_pred):
        return np.sqrt(np.mean(np.square(y_true - y_pred)))

    def forward(self, X):
        # Calcul de la sortie de la couche cachée
        hidden_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = self.sigmoid(hidden_input)
        # Calcul de la sortie finale
        output_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        predicted_output = self.sigmoid(output_input)
        return predicted_output

    def backward(self, X, y, predicted_output, learning_rate):
        # Calcul de l'erreur
        error = y - predicted_output
        # Calcul du gradient de l'erreur par rapport à la sortie
        delta_output = error * self.sigmoid_derivative(predicted_output)
        # Calcul du gradient de l'erreur par rapport à la couche cachée
        error_hidden = np.dot(delta_output, self.weights_hidden_output.T)
        delta_hidden = error_hidden * self.sigmoid_derivative(self.hidden_output)
        # Mise à jour des poids et des biais
        self.weights_hidden_output += np.dot(self.hidden_output.T, delta_output) * learning_rate
        self.weights_input_hidden += np.dot(X.T, delta_hidden) * learning_rate
        self.bias_output += np.sum(delta_output, axis=0) * learning_rate
        self.bias_hidden += np.sum(delta_hidden, axis=0) * learning_rate

    def train(self, X, y, epochs, learning_rate):
        for epoch in range(epochs):
            # Passage avant
            predicted_output = self.forward(X)
            # Passage arrière
            self.backward(X, y, predicted_output, learning_rate)
            # Calcul de la perte (loss)
            loss = self.rmse_loss(y, predicted_output)
            if epoch % 100 == 0:
                print(f'Epoch {epoch}, Loss: {loss:.4f}')

# Exemple d'utilisation
# Supposons que X soit vos caractéristiques et y soit votre colonne cible (la taille de la population)

input_size = X_train.shape[1]
hidden_size = 100  # Taille de la couche cachée
output_size = 1  # Taille de la sortie
learning_rate = 0.1
epochs = 1000

# Initialisation du réseau de neurones
nn = NeuralNetwork(input_size, hidden_size, output_size)

# Entraînement du réseau de neurones
nn.train(X_train, y_train, epochs, learning_rate)

# Faire des prédictions
predictions_2 = nn.forward(X_pred)
print("Prédictions:")
print(predictions_2)

In [ ]:
df_pred_2 = unique_combinations.copy()
df_pred_2['predictions'] = predictions_2
df_pred_2['predictions'] = df_pred_2['predictions'].astype(int)
df_merged_2 = pd.merge(df_pred_2, df_try[['country_encoded', 'Country']], on='country_encoded', how='left').drop_duplicates().reset_index(drop=True)
df_merged_2

In [ ]:
### Prédictions Random Forest

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Entraînement du modèle
rf_model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
predictions_test = rf_model.predict(X_test)

# Calcul de la RMSE (Root Mean Squared Error)
rmse = mean_squared_error(y_test, predictions_test, squared=False)

# Predictions
predictions_3 = rf_model.predict(X_pred)

print("RMSE:", rmse)

In [ ]:
df_pred_3 = unique_combinations.copy()
df_pred_3['predictions'] = predictions_3
df_pred_3['predictions'] = df_pred_3['predictions'].astype(int)
df_merged_3 = pd.merge(df_pred_3, df_try[['country_encoded', 'Country']], on='country_encoded', how='left').drop_duplicates().reset_index(drop=True)
df_merged_3